In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split as tt
from sklearn.metrics import accuracy_score as acc
%matplotlib inline

In [156]:
file = open("dummy.csv", "r")
file1 = open("data_nn.csv", "w")
file1.write('"Ping Local" "Ping LocalApp-CorpApp" "Ping LocalApp-CorpDB" "Load Average LocalApp" "Load Average LocalDB" "Load Average CORPApp" "TNS ping LocalApp-LocalDB" "TNS ping LocalApp-CorpDB" "Class0" "Class1" "Class2" "Class3"\n')
for line in file:
    l = len(line)
    dat = list(line.split())
    if (dat[1] == 'x' or dat[5] == 'x'):
        line = ""
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 1 0 0\n"
    elif (dat[2] == 'x' or dat[7] == 'x'):
        line = ""
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 0 1 0\n"
    elif (dat[3] == 'x' or dat[4] == 'x' or dat[6] == 'x'):
        line = ""
        for da in dat:
            if (da == 'x'):
                da = '9999999999'
            line+=da+" "
        line += "0 0 0 1\n"
    else:
        line = line[:-1] + " 1 0 0 0" + line[-1]
    file1.write(line)
file.close()
file1.close()

In [157]:
data = pd.read_csv("data_nn.csv",delimiter = ' ')
x = data.ix[:,:8]
y = data.ix[:,8:]
x_train,x_test,y_train,y_test = tt(x,y,test_size=0.2,random_state=1)

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [158]:
inputX = x_train.as_matrix()
inputY = y_train.as_matrix()

In [159]:
inputY

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ..., 
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0]], dtype=int64)

In [160]:
learning_rate = 0.02
training_epochs = 20000
display_step = 1000
n_samples = inputY.size

In [161]:
#computation graph
x = tf.placeholder(tf.float32,[None,8])
#weights
W = tf.Variable(tf.random_normal([8,4]))

#biases
b = tf.Variable(tf.random_normal([4]))

#calculations
y_values = tf.add(tf.matmul(x,W),b)

y = tf.nn.softmax(y_values)

y_ = tf.placeholder(tf.float32,[None,4])


In [162]:
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [163]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [164]:
#print ("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')
for i in range(training_epochs):  
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY}) # Take a gradient descent step using our inputs and labels
    
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print ("Training step:", '%04d' % (i), "cost=", "{:.9f}".format(cc)) #, \"W=", sess.run(W), "b=", sess.run(b)

print ("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print ("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')


Training step: 0000 cost= 0.249645978
Training step: 1000 cost= 0.237485602
Training step: 2000 cost= 0.237446755
Training step: 3000 cost= 0.237215415
Training step: 4000 cost= 0.236538902
Training step: 5000 cost= 0.236251697
Training step: 6000 cost= 0.236251563
Training step: 7000 cost= 0.236251459
Training step: 8000 cost= 0.236251339
Training step: 9000 cost= 0.236250713
Training step: 10000 cost= 0.231770769
Training step: 11000 cost= 0.076663874
Training step: 12000 cost= 0.075860836
Training step: 13000 cost= 0.075812437
Training step: 14000 cost= 0.075811841
Training step: 15000 cost= 0.075811706
Training step: 16000 cost= 0.075811639
Training step: 17000 cost= 0.075811595
Training step: 18000 cost= 0.075811557
Training step: 19000 cost= 0.075811535
Optimization Finished!
Training cost= 0.0758115 W= [[-0.22596018 -1.33724296  0.06848895  0.22809279]
 [ 2.07356644  0.27426183 -3.1072793   0.08429413]
 [-0.60161954  0.62087625 -0.47843215  0.56332141]
 [-0.43587506  1.16097975 

In [165]:
test_x = x_test.as_matrix()
result = sess.run(y, feed_dict={x: x_test})
A = np.asarray(result).reshape(-1)
test_y = y_test.as_matrix()
B = np.asarray(test_y).reshape(-1)
print (str(((A==B).sum()/A.size)*100)+"% accuracy ")
np.savetxt("output_nn.csv", result , delimiter=" ",fmt= "%0.3f")

79.7794117647% accuracy 
